In [71]:
import os
import time
import tempfile
import warnings
from os.path import join as pjoin

import dask
import dask.dataframe as dd
import dask_geopandas as dgpd
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio
from scipy.fft import dst
import tqdm
import xarray as xr
from dask.diagnostics import ProgressBar
from rasterio.crs import CRS

from raster_tools import Raster, Vector, open_vectors, clipping, zonal
from raster_tools.dtypes import F32, U8, U16

# Filter out warnings from dask_geopandas and dask
warnings.filterwarnings(
    "ignore", message=".*initial implementation of Parquet.*"
)
warnings.filterwarnings(
    "ignore", message=".*Slicing is producing a large chunk.*"
)


# Location for temporary storage
TMP_LOC = "/home/jake/FireLab/Project/data/temp/"
TMP_LOC2 = "/home/jake/FireLab/Project/data/temp2/"
DATA_LOC = "/home/jake/FireLab/Project/data/"

STATE = "OR"

# Location of clipped DEM files
DEM_DATA_DIR = pjoin(TMP_LOC, "dem_data")

# location of feature data files
FEATURE_DIR = pjoin(DATA_LOC, "FeatureData")
EDNA_DIR = pjoin(DATA_LOC, "terrain")
MTBS_DIR = pjoin(DATA_LOC, "MTBS_Data")
VIIRS_DIR = pjoin(DATA_LOC, "viirs_data")

PATHS = {
    "states": pjoin(EDNA_DIR, "state_borders/cb_2018_us_state_5m.shp"),
    "dem": pjoin(EDNA_DIR, "us_orig_dem/us_orig_dem/orig_dem/hdr.adf"),
    "dem_slope": pjoin(EDNA_DIR, "us_slope/us_slope/slope/hdr.adf"),
    "dem_aspect": pjoin(EDNA_DIR, "us_aspect/aspect/hdr.adf"),
    "dem_flow_acc": pjoin(EDNA_DIR, "us_flow_acc/us_flow_acc/flow_acc/hdr.adf"),
    "gm_srad": pjoin(FEATURE_DIR, "gridmet/srad_1986_2020_weekly.nc"),
    "gm_vpd": pjoin(FEATURE_DIR, "gridmet/vpd_1986_2020_weekly.nc"),
    "aw_mat": pjoin(FEATURE_DIR, "adaptwest/Normal_1991_2020_MAT.tif"),
    "aw_mcmt": pjoin(FEATURE_DIR, "adaptwest/Normal_1991_2020_MCMT.tif"),
    "aw_mwmt": pjoin(FEATURE_DIR, "adaptwest/Normal_1991_2020_MWMT.tif"),
    "aw_td": pjoin(FEATURE_DIR, "adaptwest/Normal_1991_2020_TD.tif"),
    "dm_tmax": pjoin(FEATURE_DIR, "daymet/tmax_1986_2020.nc"),
    "dm_tmin": pjoin(FEATURE_DIR, "daymet/tmin_1986_2020.nc"),
    "biomass_afg": pjoin(
        FEATURE_DIR, "biomass/biomass_afg_1986_2020_{}.nc".format(STATE)
    ),
    "biomass_pfg": pjoin(
        FEATURE_DIR, "biomass/biomass_pfg_1986_2020_{}.nc".format(STATE)
    ),
    "landfire_fvt": pjoin(
        FEATURE_DIR, "landfire/LF2020_FVT_200_CONUS/Tif/LC20_FVT_200.tif"
    ),
    "landfire_fbfm40": pjoin(
        FEATURE_DIR, "landfire/LF2020_FBFM40_200_CONUS/Tif/LC20_F40_200.tif"
    ),
    "ndvi": pjoin(FEATURE_DIR, "ndvi/access/weekly/ndvi_1986_2020_weekavg.nc"),
    "mtbs_root": pjoin(MTBS_DIR, "MTBS_BSmosaics/"),
    "mtbs_perim": pjoin(MTBS_DIR, "mtbs_perimeter_data/mtbs_perims_DD.shp"),
    "viirs_root": VIIRS_DIR,
    "viirs_perim": pjoin(VIIRS_DIR, "viirs_perims.parquet"),
}
YEARS = list(range(2018, 2021))
GM_KEYS = list(filter(lambda x: x.startswith("gm_"), PATHS))
AW_KEYS = list(filter(lambda x: x.startswith("aw_"), PATHS))
DM_KEYS = list(filter(lambda x: x.startswith("dm_"), PATHS))
BIOMASS_KEYS = list(filter(lambda x: x.startswith("biomass_"), PATHS))
LANDFIRE_KEYS = list(filter(lambda x: x.startswith("landfire_"), PATHS))
NDVI_KEYS = list(filter(lambda x: x.startswith("ndvi"), PATHS))
DEM_KEYS = list(filter(lambda x: x.startswith("dem"), PATHS))

# NC_KEYSET = set(GM_KEYS + DM_KEYS + BIOMASS_KEYS + NDVI_KEYS)
NC_KEYSET = [DM_KEYS, GM_KEYS, BIOMASS_KEYS, NDVI_KEYS]
TIF_KEYSET = [AW_KEYS, LANDFIRE_KEYS]

MTBS_DF_PATH = pjoin(TMP_LOC, f"{STATE}_mtbs.parquet")
MTBS_DF_PARQUET_PATH_NEW = pjoin(TMP_LOC, f"{STATE}_mtbs_new.parquet")
MTBS_DF_TEMP_PATH = pjoin(TMP_LOC, f"{STATE}_mtbs_temp.parquet")
MTBS_DF_TEMP_PATH_2 = pjoin(TMP_LOC, f"{STATE}_mtbs_temp_2.parquet")
CHECKPOINT_1_PATH = pjoin(TMP_LOC, "check1")
CHECKPOINT_2_PATH = pjoin(TMP_LOC, "check2")


def hillshade(slope, aspect, azimuth=180, zenith=45):
    # Convert angles from degrees to radians
    azimuth_rad = np.radians(azimuth)
    zenith_rad = np.radians(zenith)
    slope_rad = np.radians(slope)
    aspect_rad = np.radians(aspect)

    # Calculate hillshade
    shaded = np.sin(zenith_rad) * np.sin(slope_rad) + \
             np.cos(zenith_rad) * np.cos(slope_rad) * \
             np.cos(azimuth_rad - aspect_rad)
    # scale to 0-255
    shaded = 255 * (shaded + 1) / 2
    # round hillshade to nearest integer
    shaded = np.rint(shaded)
    # convert to uint8
    # Ensure non-finite values are not converted to int
    # shaded = np.where(np.isfinite(shaded), shaded.astype(np.uint8), np.nan)
    return shaded

def hillshade_partition(df, zenith, azimuth):
    # Apply the hillshade function to the slope and aspect columns
    df['hillshade'] = hillshade(df['dem_slope'], df['dem_aspect'], azimuth, zenith)
    return df

def timestamp_to_year_part(df):
    # Assuming 'ig_date' is the column with timestamp data
    df['year'] = df['ig_date'].dt.year
    return df

def get_nc_var_name(ds, nc_feat_name):
    # Find the data variable in a nc xarray.Dataset
    if nc_feat_name.startswith("dm"):
        var_name = list(set(ds.keys()) - set(["crs", "bnds"]))[0] # for DAYMET ONLY!!
    else:
        var_name = list(set(ds.keys()) - set(["crs", "day_bnds"]))[0]
    return var_name


def netcdf_to_raster(path, date, nc_feature_name):
    # This produces a Dataset. We need to grab the DataArray inside that
    # contains the data of interest.
    nc_ds = xr.open_dataset(path, chunks={"day": 1})#, decode_times=False)
    if nc_feature_name == "ndvi":
        nc_ds2 = nc_ds.drop_vars(
        ["latitude_bnds", "longitude_bnds", "time_bnds"]
        ).rio.write_crs("EPSG:5071") # FOR NDVI ONLY!!
    elif nc_feature_name.startswith("dm_"):
        nc_ds2 = nc_ds.rio.write_crs("EPSG:5071")  # FOR DAYMET ONLY!!
        # nc_ds = nc_ds.rio.write_crs(
        #     nc_ds.coords["lambert_conformal_conic"].spatial_ref
        # )  # FOR DAYMET ONLY!!
        nc_ds2 = nc_ds2.rename({"lambert_conformal_conic": "crs"})  # FOR DAYMET ONLY!!
        nc_ds2 = nc_ds2.drop_vars(["lat", "lon", "time_bnds"])  # FOR DAYMET ONLY!!
        nc_ds = None # FOR DAYMET ONLY!!
        nc_ds2 = nc_ds2.rename_vars({"x": "lon", "y": "lat"})  # FOR DAYMET ONLY!!
    else:
        nc_ds2 = nc_ds.rio.write_crs("EPSG:5071")

    # Find variable name
    var_name = get_nc_var_name(nc_ds2, nc_feature_name)
    # print(f"var_name: {var_name}")
    # Extract
    var_da = nc_ds2[var_name]
    # print(f"{var_da = }")
    if nc_feature_name.startswith("gm_"):
        var_da = var_da.sel(day=date, method="nearest") # for GM
    else:
        var_da = var_da.sel(time=date, method="nearest") # for DM and BM and NDVI
    if nc_feature_name.startswith("ndvi"):
        xrs = xr.DataArray(
            var_da.data, dims=("y", "x"), coords=(var_da.latitude.data, var_da.longitude.data)
        ).expand_dims("band") # FOR NDVI ONLY!!
    else:
        xrs = xr.DataArray(
            var_da.data, dims=("y", "x"), coords=(var_da.lat.data, var_da.lon.data)
        ).expand_dims("band") # For non-NDVI
    xrs["band"] = [1]
    # Set CRS in raster compliant format
    xrs = xrs.rio.write_crs(nc_ds2.crs.spatial_ref)
    return Raster(xrs)


def extract_nc_data(df, nc_name):
    assert df.ig_date.unique().size == 1
    # print(f"{gm_name}: {df.columns = }, {len(df) = }")
    date = df.ig_date.values[0]
    print(f"{nc_name}: starting {date}")
    rs = netcdf_to_raster(PATHS[nc_name], date, nc_name)
    bounds = gpd.GeoSeries(df.geometry).to_crs(rs.crs).total_bounds
    rs = clipping.clip_box(rs, bounds)
    if type(df) == pd.DataFrame:
        df = gpd.GeoDataFrame(df)
    feat = Vector(df, len(df))
    rdf = (
        zonal.extract_points_eager(feat, rs, skip_validation=True)
        .drop(columns=["band"])
        .rename(columns={"extracted": nc_name})
        .compute()
    )
    df[nc_name].values[:] = rdf[nc_name].values
    # print(f"{nc_name}: finished {date}")
    return df


def get_state_dem_path(dem_key, state):
    return pjoin(DEM_DATA_DIR, f"{state}_{dem_key}.tif")


def extract_dem_data(df, key):
    state = df.state.values[0]
    path = get_state_dem_path(key, state)
    rs = Raster(path)
    if type(df) == pd.DataFrame:
        df = gpd.GeoDataFrame(df)
    feat = Vector(df, len(df))
    rdf = (
        zonal.extract_points_eager(feat, rs, skip_validation=True)
        .drop(columns=["band"])
        .compute()
    )
    df[key].values[:] = rdf.extracted.values
    return df

def extract_tif_data(df, key):
    state = df.state.values[0]
    path = PATHS[key]
    rs = Raster(path)
    if type(df) == pd.DataFrame:
        df = gpd.GeoDataFrame(df)
    feat = Vector(df, len(df))
    rdf = (
        zonal.extract_points_eager(feat, rs, skip_validation=True)
        .drop(columns=["band"])
        .compute()
    )
    df[key].values[:] = rdf.extracted.values
    return df


def partition_extract_nc(df, key):
    # This func wraps extract_nc_data. It groups the partition in to sub
    # dataframes with the same date and then applies extract_nc_data to
    # each and reassembles the results into an output dataframe.
    parts = []
    for group in df.groupby("ig_date", sort=True):
        _, gdf = group
        parts.append(extract_nc_data(gdf, key))
    return pd.concat(parts)

def partition_extract_tif(df, key):
    # This func wraps extract_tif_data. It groups the partition in to sub
    # dataframes with the same date and then applies extract_tif_data to
    # each and reassembles the results into an output dataframe.
    parts = []
    for group in df.groupby("ig_date", sort=True):
        _, gdf = group
        parts.append(extract_tif_data(gdf, key))
    return pd.concat(parts)

def clip_and_save_dem_rasters(keys, paths, feature, state):
    feature = feature.compute()
    for k in tqdm.tqdm(keys, ncols=80, desc="DEM Clipping"):
        path = paths[k]
        out_path = get_state_dem_path(k, state)
        if os.path.exists(out_path):
            continue
        rs = Raster(path)
        (bounds,) = dask.compute(feature.to_crs(rs.crs).total_bounds)
        crs = clipping.clip_box(rs, bounds)
        crs.save(out_path)


def build_mtbs_year_df(perims_df, state_label):
    dfs = []
    for grp in perims_df.groupby("Ig_Date"):
        date, perim = grp
        perim = perim.copy()
        perim["ig_date"] = date
        perim["state"] = state_label
        df = perim.assign(
            ig_date=lambda frame: dd.to_datetime(
                frame.ig_date, format="%Y-%m-%d"
            ),
            state=lambda frame: frame.state.astype("category"),
            duration=lambda frame: frame.duration.astype("float32"),
            buffer=lambda frame: frame.buffer,
            fireid=lambda frame: frame.fireid
        )
        dfs.append(df)
    return dd.concat(dfs)


def _build_mtbs_df(
    years, year_to_perims, state, working_dir
):
    dfs = []
    it = tqdm.tqdm(years, ncols=80, desc="MTBS")
    for y in it:
        # mtbs_path = year_to_mtbs_file[y]
        # if not os.path.exists(mtbs_path):
        #     it.write(f"No data for {y}")
        #     continue
        perims = year_to_perims[y]
        ydf = build_mtbs_year_df(perims, state)
        ypath = pjoin(working_dir, str(y))
        ydf.compute().to_parquet(ypath)
        ydf = dgpd.read_parquet(ypath)
        dfs.append(ydf)
    return dd.concat(dfs)


def build_mtbs_df(
    years, year_to_perims, state, out_path, tmp_loc=TMP_LOC
):
    print("Building mtbs df")
    with tempfile.TemporaryDirectory(dir=tmp_loc) as working_dir:
        df = _build_mtbs_df(
            years, year_to_perims, state, working_dir
        )
        with ProgressBar():
            df.to_parquet(out_path)
    return dgpd.read_parquet(out_path)


def add_columns_to_df(
    df,
    columns,
    part_func,
    out_path,
    col_type=F32,
    col_default=np.nan,
    part_func_args=(),
    # tmp_loc=TMP_LOC,
    tmp_loc=TMP_LOC2,
    parallel=True,
):
    print(f"Adding columns: {columns}")
    print('Existing columns: ', df.columns)
    # Add columns
    expanded_df = df.assign(**{c: col_type.type(col_default) for c in columns})
    # show expanded_df
    print('Expanded columns: ', expanded_df.columns)
    # with tempfile.TemporaryDirectory(dir=tmp_loc) as working_dir:
    #     # Save to disk before applying partition function. to_parquet() has a
    #     # chance of segfaulting and that chance goes WAY up after adding
    #     # columns and then mapping a function to partitions. Saving to disk
    #     # before mapping keeps the odds low.
    #     path = pjoin(working_dir, "expanded")
    #     print(expanded_df.head())
    #     # reset the index to fix keyerror not in index
    #     expanded_df = expanded_df.reset_index()
    #     expanded_df.to_parquet(path)

    #     expanded_df = dgpd.read_parquet(path)
    #     meta = expanded_df._meta.copy()
    #     for c in columns:
    #         expanded_df = expanded_df.map_partitions(
    #             part_func, c, *part_func_args, meta=meta
    #         )

    #     if parallel:
    #         with ProgressBar():
    #             expanded_df.to_parquet(out_path)
    #     else:
    #         # Save parts in serial and then assemble into single dataframe
    #         with tempfile.TemporaryDirectory(dir=tmp_loc) as part_dir:
    #             dfs = []
    #             for i, part in enumerate(expanded_df.partitions):
    #                 # Save part i
    #                 part_path = pjoin(part_dir, f"part{i:04}")
    #                 with ProgressBar():
    #                     part.compute().to_parquet(part_path)
    #                 # Save paths for opening with dask_geopandas later. Avoid
    #                 # opening more dataframes in this loop as doing so will
    #                 # likely cause a segfault. I have no idea why.
    #                 dfs.append(part_path)
    #             dfs = [dgpd.read_parquet(p) for p in dfs]
    #             # Assemble and save to final output location
    #             expanded_df = dd.concat(dfs)
    #             with ProgressBar():
    #                 expanded_df.to_parquet(out_path)
                    ########################################
    # Save to disk before applying partition function. to_parquet() has a
    # chance of segfaulting and that chance goes WAY up after adding
    # columns and then mapping a function to partitions. Saving to disk
    # before mapping keeps the odds low.
    working_dir = tmp_loc
    path = pjoin(working_dir, "expanded")
    # show the type of expanded_df
    print('expanded_df type: ', type(expanded_df))
    print(expanded_df.head())
    # reset the index to fix keyerror not in index
    print(expanded_df)
    expanded_df = expanded_df.reset_index()
    print(expanded_df)
    print(path)
    expanded_df.to_parquet(path)
    expanded_df = dgpd.read_parquet(path)
    meta = expanded_df._meta.copy()
    for c in columns:
        expanded_df = expanded_df.map_partitions(
            part_func, c, *part_func_args, meta=meta
        )
    if parallel:
        with ProgressBar():
            expanded_df.to_parquet(out_path)
    else:
        # Save parts in serial and then assemble into single dataframe
        with tempfile.TemporaryDirectory(dir=tmp_loc) as part_dir:
            dfs = []
            for i, part in enumerate(expanded_df.partitions):
                # Save part i
                part_path = pjoin(part_dir, f"part{i:04}")
                with ProgressBar():
                    part.compute().to_parquet(part_path)
                # Save paths for opening with dask_geopandas later. Avoid
                # opening more dataframes in this loop as doing so will
                # likely cause a segfault. I have no idea why.
                dfs.append(part_path)
            dfs = [dgpd.read_parquet(p) for p in dfs]
            # Assemble and save to final output location
            expanded_df = dd.concat(dfs)
            with ProgressBar():
                expanded_df.to_parquet(out_path)
    return dgpd.read_parquet(out_path)

In [ ]:
### MAIN ###

if __name__ == "__main__":

    # start timer
    start = time.time()

    if 1:
        # State borders
        print("Loading state borders")
        stdf = open_vectors(PATHS["states"], 0).data.to_crs("EPSG:5071")
        states = {st: stdf[stdf.STUSPS == st].geometry for st in list(stdf.STUSPS)}
        state_shape = states[STATE]
        states = None
        stdf = None

        print("Loading VIIRS perimeters")
        perimdf = gpd.read_parquet(DATA_LOC + "viirs_perims.parquet").to_crs("EPSG:5071")
        # rename column t to Ig_Date
        perimdf = perimdf.rename(columns={"t": "Ig_Date"})
        state_fire_perims = perimdf.clip(state_shape.compute())
        state_fire_perims = (
            state_fire_perims.assign(
                Ig_Date=lambda frame: dd.to_datetime(
                    frame.Ig_Date, format="%Y-%m-%d"
                )
            )
            .sort_values("Ig_Date")
        )
        state_fire_perims = state_fire_perims[state_fire_perims.Ig_Date.dt.year.between(2018, 2020)]
        year_to_perims = {
            y: state_fire_perims[state_fire_perims.Ig_Date.dt.year == y]
            for y in YEARS
        }

        # print(year_to_mtbs_file)


    if 0:
        # code below for creating a new dataset for a new state / region
        df = build_mtbs_df(
            YEARS,
            year_to_perims=year_to_perims,
            state=STATE,
            out_path=MTBS_DF_TEMP_PATH_2,
        )
        clip_and_save_dem_rasters(DEM_KEYS, PATHS, state_shape, STATE)
        df = add_columns_to_df(
            df,
            DEM_KEYS,
            extract_dem_data,
            CHECKPOINT_1_PATH,
            # Save results in serial to avoid segfaulting. Something about the
            # dem computations makes segfaults extremely likely when saving
            # The computations require a lot of memory which may be what
            # triggers the fault.
            parallel=False,
        )
        df = df.repartition(partition_size="100MB").reset_index(drop=True)
        print("Repartitioning")
        with ProgressBar():
            df.to_parquet(CHECKPOINT_2_PATH)
        df = None

    if 0:
        # code below used to add new features to the dataset
        with ProgressBar():
            df = dgpd.read_parquet(CHECKPOINT_2_PATH)

        # loop to add all nc features
        for nc_name in NC_KEYSET:
            print(f"Adding {nc_name}")
            df = add_columns_to_df(
                df, nc_name, partition_extract_nc, CHECKPOINT_1_PATH, parallel=False
            )
        # loop to add all tif features
        # for tif_name in TIF_KEYSET:
        #     print(f"Adding {tif_name}")
        #     df = add_columns_to_df(
        #         df, [tif_name], partition_extract_tif, CHECKPOINT_1_PATH, parallel=False
        #     )
        # add hillshade and year columns
        df_meta = df._meta.copy()
        df = df.assign(hillshade=U8.type(0))
        df = df.map_partitions(hillshade_partition, 45, 180, meta=df_meta)
        df = df.assign(year=U16.type(0))
        df = df.map_partitions(timestamp_to_year_part, meta=df_meta)
        print("Columns: ", df.columns)
        df = df.repartition(partition_size="100MB").reset_index(drop=True)
        print("Repartitioning")
        df = df.compute()
        with ProgressBar():
            df.to_parquet(MTBS_DF_TEMP_PATH)

    # end timer
    end = time.time()
    print(f"Time elapsed: {end - start} seconds")

In [72]:
perimdf = gpd.read_parquet(PATHS["viirs_perim"]).to_crs("EPSG:5071")
perimdf.drop(columns=["buffer"], inplace=True)

In [82]:
perimdf

,geometry,duration,fireid,t
1,"POLYGON ((-2040844.099 2451920.709, -2040826.0...",0.0,F4734,2018-04-24 12:00:00
2,"POLYGON ((-2009343.166 2399585.893, -2009341.1...",0.0,F4693,2018-04-24 12:00:00
3,"POLYGON ((-2009067.097 2399902.223, -2009053.3...",1.0,F4693,2018-04-25 12:00:00
4,"POLYGON ((-2041844.911 2451280.637, -2041844.9...",1.0,F4734,2018-04-25 12:00:00
5,"POLYGON ((-2041844.983 2451282.932, -2041845.2...",1.5,F4734,2018-04-26 00:00:00
...,...,...,...,...
3530,"POLYGON ((-2020525.492 2600622.673, -2020521.8...",0.0,F1771,2021-12-07 12:00:00
3531,"POLYGON ((-2020521.891 2600621.213, -2020517.0...",0.5,F1771,2021-12-08 00:00:00
3532,"POLYGON ((-2022539.563 2596795.749, -2022538.7...",6.5,F1529,2021-12-09 00:00:00
3533,"POLYGON ((-2020814.395 2600143.764, -2023466.9...",1.5,F1771,2021-12-09 00:00:00


In [89]:
# Create a new DataFrame excluding the datetime column
df_without_datetime = perimdf[['geometry', 'duration', 'fireid']]

# Write this DataFrame to a shapefile
df_without_datetime.to_file("viirs_without_datetime.shp")

# Create another DataFrame with only id and datetime columns
df_with_datetime = perimdf[['fireid', 't', 'duration']]

# Write this DataFrame to a CSV file
df_with_datetime.to_csv("viirs_with_datetime.csv", index=False)

In [90]:
# Read the shapefile and CSV file
df_without_datetime = gpd.read_file("viirs_without_datetime.shp")
df_with_datetime = pd.read_csv("viirs_with_datetime.csv")

# Merge the two DataFrames on 'fireid'
merged_df = df_without_datetime.merge(df_with_datetime, on=['fireid', 'duration'])

In [94]:
merged_df

,duration,fireid,geometry,t
0,0.0,F4734,"POLYGON ((-2040844.099 2451920.709, -2040826.0...",2018-04-24 12:00:00
1,0.0,F4693,"POLYGON ((-2009343.166 2399585.893, -2009341.1...",2018-04-24 12:00:00
2,1.0,F4693,"POLYGON ((-2009067.097 2399902.223, -2009053.3...",2018-04-25 12:00:00
3,1.0,F4734,"POLYGON ((-2041844.911 2451280.637, -2041844.9...",2018-04-25 12:00:00
4,1.5,F4734,"POLYGON ((-2041844.983 2451282.932, -2041845.2...",2018-04-26 00:00:00
...,...,...,...,...
3500,0.0,F1771,"POLYGON ((-2020525.492 2600622.673, -2020521.8...",2021-12-07 12:00:00
3501,0.5,F1771,"POLYGON ((-2020521.891 2600621.213, -2020517.0...",2021-12-08 00:00:00
3502,6.5,F1529,"POLYGON ((-2022539.563 2596795.749, -2022538.7...",2021-12-09 00:00:00
3503,1.5,F1771,"POLYGON ((-2020814.395 2600143.764, -2023466.9...",2021-12-09 00:00:00


In [79]:
# read parquets from /home/jake/FireLab/Project/data/temp2/expanded  
parq0 = dgpd.read_parquet('/home/jake/FireLab/Project/data/temp2/expanded/part.0.parquet')
parq1 = dgpd.read_parquet('/home/jake/FireLab/Project/data/temp2/expanded/part.1.parquet')
parq2 = dgpd.read_parquet('/home/jake/FireLab/Project/data/temp2/expanded/part.2.parquet')


In [80]:
parq0.compute()

,index,geometry,duration,fireid,Ig_Date,buffer,ig_date,state,dem,dem_slope,dem_aspect,dem_flow_acc
0,0,"MULTIPOLYGON (((-2237727.741 2438052.370, -223...",0.5,F13066,2018-04-24 12:00:00,None,2018-04-24 12:00:00,OR,NaN,NaN,NaN,NaN
1,1,"MULTIPOLYGON (((-2248567.723 2441409.072, -224...",1.0,F13066,2018-04-24 12:00:00,None,2018-04-24 12:00:00,OR,NaN,NaN,NaN,NaN
2,0,"MULTIPOLYGON (((-2248563.416 2441416.097, -224...",1.5,F13066,2018-04-25 12:00:00,None,2018-04-25 12:00:00,OR,NaN,NaN,NaN,NaN
3,1,"MULTIPOLYGON (((-2248563.416 2441416.097, -224...",2.0,F13066,2018-04-25 12:00:00,None,2018-04-25 12:00:00,OR,NaN,NaN,NaN,NaN
4,0,"MULTIPOLYGON (((-2249476.710 2441401.638, -224...",3.0,F13066,2018-04-26 00:00:00,None,2018-04-26 00:00:00,OR,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
863,0,"MULTIPOLYGON (((-1962934.919 2405737.022, -196...",15.0,F18217,2018-12-12 12:00:00,None,2018-12-12 12:00:00,OR,NaN,NaN,NaN,NaN
864,1,"MULTIPOLYGON (((-1960095.132 2402700.336, -196...",4.5,F16943,2018-12-12 12:00:00,None,2018-12-12 12:00:00,OR,NaN,NaN,NaN,NaN
865,2,"MULTIPOLYGON (((-1960088.621 2402704.261, -196...",4.0,F16943,2018-12-12 12:00:00,None,2018-12-12 12:00:00,OR,NaN,NaN,NaN,NaN
866,0,"POLYGON ((-1960096.199 2402699.829, -1960097.2...",5.0,F16943,2018-12-13 12:00:00,None,2018-12-13 12:00:00,OR,NaN,NaN,NaN,NaN


In [ ]:
# drop buffer column
perimdf = perimdf.drop(columns=['buffer'])

In [ ]:
# from perimdf2018, plot fireid F4734 at every duration step
perimdf2018_f4734 = perimdf2018[perimdf2018.fireid == 'F4734']
perimdf2018_f4734 = perimdf2018_f4734.sort_values(by=['duration'])

duration_keys = perimdf2018_f4734.duration.unique()

for i in range(len(duration_keys)):
    # plot the fireid F4734 at each duration step
    perimdf2018_f4734_duration = perimdf2018_f4734[perimdf2018_f4734.duration == duration_keys[i]]
    perimdf2018_f4734_duration.plot()